In [254]:
import os
import json
import pandas as pd
import traceback

In [255]:
from dotenv import load_dotenv
load_dotenv()

True

In [256]:
KEY = os.getenv("GOOGLEPALM_API_KEY")

In [257]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=KEY)

In [258]:
llm

ChatGoogleGenerativeAI(model='gemini-1.5-pro', google_api_key=SecretStr('**********'), client=genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
))

In [259]:
from langchain.chat_models import ChatGooglePalm
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [260]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    }
}

In [261]:
TEMPLATE = """
Text:{text}
You are an expert mcq maker. Given above the text, it is your job to \
create a quiz of {number} multiple questions for {subject} students in {tone} tone.
Make sure to format your response like RESPONSE_JSON below and use it as guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [262]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number","subject","tone","response_json"],
    template=TEMPLATE
)

In [263]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt,output_key="quiz",verbose=True)

In [264]:
TEMPLATE2 = """
You are an expert grammerian and writer. Given a multiple choice question for {subject} students.\
you need to evaluate the complexicity of the question and give a complete analysis of the quiz. only use at max 50 words for complexicity
if the quiz is not at par with the cognitive and analytical abilities of the student,\
update the quiz questions which needs to be changed and the change the tone such that it perfectly fit the students ability.\
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [265]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [266]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [267]:
generate_evaluate_chain = SequentialChain(
    chains = [quiz_chain,review_chain],
    input_variables = ["text", "number","subject","tone","response_json"],
    output_variables = ["quiz","review"],
    verbose = True
)

In [268]:
file_path = 'D:\Projects\MCQ_Generator\data.txt'

In [269]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [270]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [271]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [272]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [273]:
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [274]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [275]:
print(type(response))

<class 'dict'>


In [276]:
quiz = response.get("quiz")

In [277]:
quiz_string = quiz.replace("```json", "").replace("```", "").strip()

In [278]:
print(quiz_string)

{
"1": {
"mcq": "Who coined the term 'machine learning'?",
"options": {
"a": "Alan Turing",
"b": "Donald Hebb",
"c": "Arthur Samuel",
"d": "Tom M. Mitchell"
},
"correct": "c"
},
"2": {
"mcq": "What was the focus of early machine learning research in the 1960s?",
"options": {
"a": "Stock market prediction",
"b": "Cancer detection",
"c": "Pattern recognition",
"d": "Natural language processing"
},
"correct": "c"
},
"3": {
"mcq": "Donald Hebb's work contributed to the understanding of:",
"options": {
"a": "Computer vision",
"b": "How neurons interact",
"c": "Reinforcement learning",
"d": "Speech synthesis"
},
"correct": "b"
},
"4": {
"mcq": "What is a key difference between Alan Turing's approach and a cognitive definition of machine learning?",
"options": {
"a": "Turing focused on the ability to perform tasks, not internal thought processes.",
"b": "Turing studied human cognition, while machine learning ignores it.",
"c": "Turing rejected the idea of machines learning, focusing on pre-pr

In [281]:
quiz = json.loads(quiz_string)

In [282]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

print(f"Number of items in quiz_table_data: {len(quiz_table_data)}")

Number of items in quiz_table_data: 5


In [283]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Alan Turing | b: Donald Hebb | c: Arthur Samuel | d: Tom M. Mitchell',
  'Correct': 'c'},
 {'MCQ': 'What was the focus of early machine learning research in the 1960s?',
  'Choices': 'a: Stock market prediction | b: Cancer detection | c: Pattern recognition | d: Natural language processing',
  'Correct': 'c'},
 {'MCQ': "Donald Hebb's work contributed to the understanding of:",
  'Choices': 'a: Computer vision | b: How neurons interact | c: Reinforcement learning | d: Speech synthesis',
  'Correct': 'b'},
 {'MCQ': "What is a key difference between Alan Turing's approach and a cognitive definition of machine learning?",
  'Choices': "a: Turing focused on the ability to perform tasks, not internal thought processes. | b: Turing studied human cognition, while machine learning ignores it. | c: Turing rejected the idea of machines learning, focusing on pre-programmed knowledge. | d: Turing's work predates any notion of mach

In [285]:
quiz = pd.DataFrame(quiz_table_data)

In [287]:
quiz.to_csv("MachineLearning.csv",index= False)